# 01. Bias Detection in Machine Learning Models | اكتشاف التحيز في نماذج التعلم الآلي

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Unit 1: Foundations of AI Ethics** - You need to understand ethical frameworks and case studies!
- ✅ **Basic Python knowledge**: Functions, dictionaries, data manipulation
- ✅ **Basic ML knowledge**: Classification, train/test split, confusion matrices
- ✅ **Understanding of bias**: What is algorithmic bias? (from Unit 1 case studies)

**If you haven't completed these**, you might struggle with:
- Understanding why bias detection matters
- Knowing which metrics to use for bias detection
- Interpreting bias detection results

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the FIRST example in Unit 2** - it teaches you how to detect bias in AI systems!

**Why this example FIRST?**
- **Before** you can mitigate bias, you need to detect it
- **Before** you can ensure fairness, you need to measure it
- **Before** you can fix problems, you need to identify them

**Builds on**: 
- 📓 Unit 1: Foundations (ethical frameworks, case studies like COMPAS showed us bias exists!)

**Leads to**: 
- 📓 Example 2: Bias Mitigation (once we detect bias, we learn to fix it!)
- 📓 Example 3: Fair Representation (ensuring fair representation in data)
- 📓 Example 4: Bias Case Studies (analyzing real bias cases)
- 📓 Example 5: Fair AI Development (building fair AI systems)

**Why this order?**
1. Detection provides **measurement tools** (needed before mitigation)
2. Detection teaches **what bias looks like** (critical for understanding)
3. Detection shows **how to quantify fairness** (needed for all fairness work)

---

## The Story: Finding the Problem Before Fixing It | القصة: إيجاد المشكلة قبل إصلاحها

Imagine you're a doctor diagnosing a patient. **Before** you can treat an illness, you need to diagnose it - run tests, check symptoms, identify the problem. **After** diagnosis, you can prescribe the right treatment!

Same with AI bias: **Before** we can fix bias, we need to detect it - measure fairness metrics, check for disparities, identify where bias exists. **After** detection, we can apply the right mitigation strategies!

---

## Why Bias Detection Matters | لماذا يهم اكتشاف التحيز؟

Bias detection is essential for ethical AI:
- **Identify Problems**: Find where bias exists in your models
- **Measure Fairness**: Quantify how fair (or unfair) your system is
- **Track Progress**: Monitor if bias mitigation efforts are working
- **Ensure Compliance**: Meet fairness requirements and regulations
- **Build Trust**: Demonstrate commitment to fairness

## Learning Objectives | أهداف التعلم
1. Understand different types of bias in ML models
2. Learn fairness metrics (demographic parity, equalized odds)
3. Detect bias using statistical measures
4. Visualize bias in model predictions
5. Interpret bias detection results
6. Understand when to use different fairness metrics

In [ ]:
# Step 1: Import necessary libraries
# These libraries help us detect bias in machine learning models

import matplotlib.pyplot as plt  # For creating visualizations: Charts, graphs, bias visualizations
import numpy as np  # For numerical operations: Arrays, calculations, random number generation
import pandas as pd  # For data manipulation: DataFrames, data analysis
from sklearn.model_selection import train_test_split  # For splitting data: Separate training and testing sets
from sklearn.ensemble import RandomForestClassifier  # For ML model: Classification algorithm
from sklearn.metrics import confusion_matrix, classification_report  # For model evaluation: Performance metrics
import seaborn as sns  # For statistical visualizations: Heatmaps, advanced plots
import os  # For file operations: Saving images

# Configure matplotlib settings: Set default figure size and font size for better visualizations
plt.rcParams['font.size'] = 10  # Font size: Make text readable (10pt is good for most displays)
plt.rcParams['figure.figsize'] = (14, 8)  # Figure size: 14 inches wide, 8 inches tall (good for detailed charts)

print(" Libraries imported successfully!")
print("\n📚 What each library does:")
print("   - matplotlib/seaborn: Create visualizations (bias charts, heatmaps)")
print("   - numpy: Numerical operations (arrays, calculations)")
print("   - pandas: Data manipulation (DataFrames, analysis)")
print("   - sklearn: Machine learning (models, metrics, data splitting)")
print("   - os: File operations (saving images)")


In [ ]:
# Step 2: Generate synthetic data with intentional bias
# This creates a dataset where we know bias exists, so we can practice detecting it

# BEFORE: No data with known bias to practice on
# AFTER: We'll have a synthetic hiring dataset with intentional bias

print("\n" + "="*80)
print("📊 GENERATING SYNTHETIC DATA WITH BIAS")
print("="*80)
print("\nWe'll create a hiring dataset where:")
print("  - Group_B has lower hiring rates even with similar qualifications")
print("  - This simulates real-world bias we need to detect")
print("  - We'll use this to practice bias detection methods\n")

def generate_biased_data(n_samples=2000):
    """
    Generate synthetic hiring data with inherent bias.
    
    HOW IT WORKS:
    1. Creates synthetic features (age, experience, education, skills)
    2. Introduces intentional bias: Group_B has lower hiring rates
    3. Calculates hiring probability with bias factor
    4. Creates binary hiring outcome
    
    ⏰ WHEN to use: To create test data with known bias for practice
    💡 WHY use: Allows us to practice bias detection on data where we know bias exists
    """
    # Set random seed: Ensure reproducible results
    np.random.seed(42)  # Seed value: Makes random numbers predictable for consistency
    
    # Create synthetic dataset: Generate features for hiring simulation
    # Why synthetic? Allows us to control bias and practice detection safely
    data = {
        'age': np.random.randint(22, 65, n_samples),  # Age: Random ages between 22-65
        'experience_years': np.random.randint(0, 20, n_samples),  # Experience: Years of work experience
        'education_level': np.random.choice([1, 2, 3, 4], n_samples, 
                                           p=[0.2, 0.3, 0.3, 0.2]),  # Education: 1-4 scale with probabilities
        'skill_score': np.random.normal(70, 15, n_samples),  # Skills: Normal distribution, mean=70, std=15
        'group': np.random.choice(['Group_A', 'Group_B'], n_samples, p=[0.5, 0.5])  # Group: Two groups, equal probability
    }
    df = pd.DataFrame(data)  # Create DataFrame: Convert dictionary to pandas DataFrame
    
    # Introduce bias: Group_B has lower success rates even with similar qualifications
    # Why introduce bias? To simulate real-world discrimination we need to detect
    bias_factor = np.where(df['group'] == 'Group_B', -0.15, 0)  # Bias: -0.15 penalty for Group_B
    
    # Calculate hiring probability: Base probability with bias factor
    # Why this formula? Combines qualifications (skills, experience, education) with bias
    base_prob = (df['skill_score'] / 100 +  # Skills component: Normalize to 0-1
                 df['experience_years'] / 20 +  # Experience component: Normalize to 0-1
                 df['education_level'] / 4) / 3 + bias_factor  # Education component: Average all three, add bias
    
    # Add noise: Real-world randomness
    base_prob += np.random.normal(0, 0.1, n_samples)  # Noise: Small random variation
    base_prob = np.clip(base_prob, 0, 1)  # Clip: Ensure probability stays between 0 and 1
    
    # Create binary outcome: Hired (1) or not hired (0)
    df['hired'] = (base_prob > 0.5).astype(int)  # Binary: 1 if probability > 0.5, else 0
    
    return df  # Return: DataFrame with features and biased hiring outcome

# Generate the biased dataset
print("Generating synthetic hiring data with bias...")
df = generate_biased_data(n_samples=2000)
print(f" Generated dataset with {len(df)} samples")
print(f"   Group_A hiring rate: {df[df['group']=='Group_A']['hired'].mean():.2%}")
print(f"   Group_B hiring rate: {df[df['group']=='Group_B']['hired'].mean():.2%}")
print("   (Notice the difference - this is the bias we'll detect!)")


## Part 3: Detecting Bias with Fairness Metrics | الجزء الثالث: اكتشاف التحيز باستخدام مقاييس العدالة

### 📚 Prerequisites (What You Need First)
-  **Biased data generated** (from Part 2) - Understanding the dataset with known bias
-  **Understanding of fairness** - Knowing what fairness means

### 🔗 Relationship: What This Builds On
This is where we actually detect the bias we created!
- Builds on: Biased dataset, understanding of fairness metrics
- Shows: How to measure and detect bias

### 📖 The Story
**Before detection**: We have biased data but don't know how to measure it.
**After detection**: We can quantify bias using fairness metrics and see exactly where it exists!

---

## Step 3: Calculate Fairness Metrics | الخطوة 3: حساب مقاييس العدالة

**BEFORE**: We have data but don't know how to measure bias.

**AFTER**: We'll calculate fairness metrics (demographic parity, equalized odds) to detect bias!

**Why fairness metrics?** They provide:
- Quantitative measures of bias
- Standard ways to compare groups
- Clear thresholds for what's "fair"


## Part 5: Visualizing Bias | الجزء الخامس: تصور التحيز

### 📚 Prerequisites (What You Need First)
- ✅ **Bias detection results** (from Part 4) - Having fairness metrics calculated
- ✅ **Visualization libraries** (from Part 1) - Understanding matplotlib/seaborn

### 🔗 Relationship: What This Builds On
This visualizes the bias we detected!
- Builds on: Bias detection results, visualization skills
- Shows: Visual representation of bias metrics

### 📖 The Story
**Before visualization**: We have numbers but can't easily see the bias.
**After visualization**: We can see bias clearly in charts and graphs!

---

## Step 5: Visualize Bias Detection Results | الخطوة 5: تصور نتائج اكتشاف التحيز

**BEFORE**: We have bias metrics but no visual representation.

**AFTER**: We'll create charts showing demographic parity, equalized odds, and confusion matrices!

**Why visualize?** Visual representation helps us:
- See bias patterns clearly
- Compare groups easily
- Communicate findings to others
- Identify which groups are most affected


In [ ]:
# Step 5: Create visualizations of bias detection results
# This helps us see bias patterns clearly

# BEFORE: We have numbers but no visual representation
# AFTER: We'll have clear charts showing bias metrics

print("\n" + "="*80)
print("📊 CREATING BIAS VISUALIZATIONS")
print("="*80)
print("\nWe'll create three visualizations:")
print("  1. Demographic Parity Chart: Shows prediction rates by group")
print("  2. Equalized Odds Chart: Shows TPR and FPR by group")
print("  3. Confusion Matrices: Shows prediction accuracy by group\n")

def visualize_demographic_parity(parity_rates, disparity):
    """
    Visualize demographic parity analysis.
    
    HOW IT WORKS:
    1. Create bar chart showing positive prediction rates for each group
    2. Add value labels on bars
    3. Add reference lines showing max/min rates
    4. Display disparity value
    5. Save as high-resolution image
    
    ⏰ WHEN to use: After calculating demographic parity - see visual comparison
    💡 WHY use: Bar chart makes it easy to see differences between groups
    """
    # Extract data: Get groups and their rates
    groups = list(parity_rates.keys())  # Groups: Extract group names for x-axis
    rates = list(parity_rates.values())  # Rates: Extract positive rates for bars
    colors = ['#3498db', '#e74c3c']  # Colors: Blue and red for visual distinction
    
    # Create chart: Initialize the plot
    fig, ax = plt.subplots(figsize=(10, 6))  # Create plot: 10x6 inches for readable chart
    
    # Create bars: Draw bars for each group
    bars = ax.bar(groups, rates, color=colors, alpha=0.8, edgecolor='black', linewidth=2)  # Bars: Group names on x-axis, rates on y-axis
    
    # Add value labels: Show exact rates on bars
    for bar, rate in zip(bars, rates):  # Loop through bars: Process each group
        height = bar.get_height()  # Get height: Extract bar height (rate value)
        ax.text(bar.get_x() + bar.get_width()/2., height,  # Position text: Center on top of bar
               f'{rate:.3f}\n({rate*100:.1f}%)',  # Text content: Show rate as decimal and percentage
               ha='center', va='bottom', fontweight='bold', fontsize=11)  # Text style: Centered, bold, readable
    
    # Add reference lines: Show max and min rates for comparison
    ax.axhline(y=max(rates), color='red', linestyle='--', alpha=0.5, label='Max')  # Max line: Red dashed line at highest rate
    ax.axhline(y=min(rates), color='blue', linestyle='--', alpha=0.5, label='Min')  # Min line: Blue dashed line at lowest rate
    
    # Add labels: Make chart readable
    ax.set_ylabel('Positive Prediction Rate', fontsize=12, fontweight='bold')  # Y-axis label: Describe what y-axis shows
    ax.set_title(f'Demographic Parity Analysis\nDisparity: {disparity:.3f}', 
                fontsize=14, fontweight='bold', pad=20)  # Title: Main heading with disparity value
    
    # Set axis limits: Ensure consistent scale
    ax.set_ylim(0, max(rates) * 1.2)  # Y-axis range: 0 to 20% above max rate (room for labels)
    
    # Add legend and grid: Improve readability
    ax.legend()  # Legend: Explain reference lines
    ax.grid(axis='y', alpha=0.3)  # Grid: Light gray lines help read values
    
    # Save visualization: Export as high-resolution image
    plt.tight_layout()  # Adjust layout: Prevent label cutoff
    script_dir = os.path.dirname(os.path.abspath(__file__))  # Get directory: Find notebook location
    output_path = os.path.join(script_dir, 'demographic_parity.png')  # File path: Save in notebook directory
    plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Save image: High resolution (300 dpi), tight bounds
    print(" Saved: demographic_parity.png")  # Success message: Confirm save
    plt.close()  # Close figure: Free memory

def visualize_equalized_odds(equalized_odds, tpr_disparity, fpr_disparity):
    """
    Visualize equalized odds metrics.
    
    HOW IT WORKS:
    1. Create grouped bar chart showing TPR and FPR for each group
    2. Add value labels on bars
    3. Display TPR and FPR disparity values
    4. Save as high-resolution image
    
    ⏰ WHEN to use: After calculating equalized odds - see visual comparison
    💡 WHY use: Grouped bars make it easy to compare TPR and FPR across groups
    """
    # Extract data: Get groups and their metrics
    groups = list(equalized_odds.keys())  # Groups: Extract group names
    tprs = [equalized_odds[g]['TPR'] for g in groups]  # TPRs: Extract True Positive Rates
    fprs = [equalized_odds[g]['FPR'] for g in groups]  # FPRs: Extract False Positive Rates
    
    # Set up bar positions: Create positions for grouped bars
    x = np.arange(len(groups))  # X positions: Array of positions for each group
    width = 0.35  # Bar width: Width of each bar (leaves room for two bars per group)
    
    # Create chart: Initialize the plot
    fig, ax = plt.subplots(figsize=(10, 6))  # Create plot: 10x6 inches for readable chart
    
    # Create bars: Draw grouped bars for TPR and FPR
    bars1 = ax.bar(x - width/2, tprs, width, label='True Positive Rate (TPR)',  # TPR bars: Green bars, offset left
                   color='#2ecc71', alpha=0.8, edgecolor='black')  # Color: Green for positive metric
    bars2 = ax.bar(x + width/2, fprs, width, label='False Positive Rate (FPR)',  # FPR bars: Red bars, offset right
                   color='#e74c3c', alpha=0.8, edgecolor='black')  # Color: Red for negative metric
    
    # Add value labels: Show exact values on bars
    for bars in [bars1, bars2]:  # Loop through bar groups: Process TPR and FPR bars
        for bar in bars:  # Loop through individual bars: Process each bar
            height = bar.get_height()  # Get height: Extract bar height (rate value)
            ax.text(bar.get_x() + bar.get_width()/2., height,  # Position text: Center on top of bar
                   f'{height:.3f}',  # Text content: Show rate as decimal
                   ha='center', va='bottom', fontsize=9, fontweight='bold')  # Text style: Centered, bold, readable
    
    # Add labels: Make chart readable
    ax.set_xlabel('Group', fontsize=12, fontweight='bold')  # X-axis label: Describe what x-axis shows
    ax.set_ylabel('Rate', fontsize=12, fontweight='bold')  # Y-axis label: Describe what y-axis shows
    ax.set_title(f'Equalized Odds Analysis\nTPR Disparity: {tpr_disparity:.3f} | FPR Disparity: {fpr_disparity:.3f}', 
                fontsize=14, fontweight='bold', pad=20)  # Title: Main heading with disparity values
    
    # Configure x-axis: Set group names as labels
    ax.set_xticks(x)  # Set tick positions: Place ticks at each group position
    ax.set_xticklabels(groups)  # Set tick labels: Use group names
    
    # Add legend and grid: Improve readability
    ax.legend(fontsize=10)  # Legend: Explain what TPR and FPR mean
    ax.set_ylim(0, max(max(tprs), max(fprs)) * 1.2)  # Y-axis range: 0 to 20% above max rate
    ax.grid(axis='y', alpha=0.3)  # Grid: Light gray lines help read values
    
    # Save visualization: Export as high-resolution image
    plt.tight_layout()  # Adjust layout: Prevent label cutoff
    script_dir = os.path.dirname(os.path.abspath(__file__))  # Get directory: Find notebook location
    output_path = os.path.join(script_dir, 'equalized_odds.png')  # File path: Save in notebook directory
    plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Save image: High resolution (300 dpi), tight bounds
    print(" Saved: equalized_odds.png")  # Success message: Confirm save
    plt.close()  # Close figure: Free memory

def visualize_confusion_matrices(test_df):
    """
    Visualize confusion matrices by group.
    
    HOW IT WORKS:
    1. Create separate confusion matrix for each group
    2. Use heatmap to show counts clearly
    3. Display side-by-side for easy comparison
    4. Save as high-resolution image
    
    ⏰ WHEN to use: After making predictions - see accuracy differences by group
    💡 WHY use: Confusion matrices show exactly where predictions differ between groups
    """
    # Get groups: Identify all groups in test data
    groups = test_df['group'].unique()  # Groups: Extract unique group values
    
    # Create subplots: One confusion matrix per group
    fig, axes = plt.subplots(1, len(groups), figsize=(14, 5))  # Subplots: Side-by-side, 14x5 inches
    
    # Create confusion matrix for each group: Process each group
    for idx, group in enumerate(groups):  # Loop through groups: Process each group
        group_data = test_df[test_df['group'] == group]  # Filter: Get data for this group
        cm = confusion_matrix(group_data['hired'], group_data['predicted'])  # Calculate: Confusion matrix for this group
        
        # Create heatmap: Visualize confusion matrix
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],  # Heatmap: Blue color scheme, show counts
                   cbar_kws={'label': 'Count'})  # Colorbar: Explain what colors mean
        
        # Add labels: Make heatmap readable
        axes[idx].set_title(f'{group}\nConfusion Matrix', fontsize=12, fontweight='bold')  # Title: Group name and chart type
        axes[idx].set_xlabel('Predicted', fontsize=10)  # X-axis label: Describe what x-axis shows
        axes[idx].set_ylabel('Actual', fontsize=10)  # Y-axis label: Describe what y-axis shows
        axes[idx].set_xticklabels(['Not Hired', 'Hired'])  # X labels: Meaning of columns
        axes[idx].set_yticklabels(['Not Hired', 'Hired'])  # Y labels: Meaning of rows
    
    # Save visualization: Export as high-resolution image
    plt.tight_layout()  # Adjust layout: Prevent label cutoff
    script_dir = os.path.dirname(os.path.abspath(__file__))  # Get directory: Find notebook location
    output_path = os.path.join(script_dir, 'confusion_matrices_by_group.png')  # File path: Save in notebook directory
    plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Save image: High resolution (300 dpi), tight bounds
    print(" Saved: confusion_matrices_by_group.png")  # Success message: Confirm save
    plt.close()  # Close figure: Free memory

# Create all visualizations
print("Creating bias visualizations...")
visualize_demographic_parity(parity_rates, parity_disparity)
visualize_equalized_odds(equalized_odds, tpr_disparity, fpr_disparity)
visualize_confusion_matrices(test_df)
print("\n All visualizations created!")


## 🎯 Summary: What We Learned | الملخص: ما تعلمناه

**BEFORE this notebook**: We knew bias exists but didn't know how to detect it in our models.

**AFTER this notebook**: We can:
- ✅ Generate synthetic data with known bias for practice
- ✅ Calculate fairness metrics (demographic parity, equalized odds)
- ✅ Train ML models and detect bias in their predictions
- ✅ Visualize bias using charts and confusion matrices
- ✅ Interpret bias detection results
- ✅ Understand when to use different fairness metrics

### Key Takeaways | الاستنتاجات الرئيسية

1. **Multiple Metrics Matter**: Different fairness metrics reveal different types of bias
2. **Demographic Parity vs. Equalized Odds**: They measure different aspects of fairness
3. **Bias Detection is Essential**: Must test for bias before and after model deployment
4. **Visualization Helps**: Charts make bias patterns easier to understand
5. **Systematic Approach**: Following a structured process ensures comprehensive bias detection

### Next Steps | الخطوات التالية

- 📓 **Example 2**: Bias Mitigation (learn how to fix the bias we detected!)
- 📓 **Example 3**: Fair Representation (ensure fair representation in data!)
- 📓 **Example 4**: Bias Case Studies (analyze real-world bias cases!)
- 📓 **Example 5**: Fair AI Development (build fair AI systems from the start!)

---

**Congratulations!** 🎉 You've learned how to detect bias in machine learning models systematically!


## Part 2: Understanding Bias in Data | الجزء الثاني: فهم التحيز في البيانات

### 📚 Prerequisites (What You Need First)
-  **Library imports** (from Part 1) - Understanding data manipulation and ML tools
-  **Understanding of bias** (from Unit 1) - Knowing what bias is

### 🔗 Relationship: What This Builds On
This creates data with intentional bias so we can practice detecting it!
- Builds on: Data manipulation skills, understanding of bias
- Shows: How bias manifests in data

### 📖 The Story
**Before biased data**: We need data with known bias to practice detection.
**After biased data**: We have a dataset where we know bias exists, so we can test our detection methods!
## Part 4: Training Model and Detecting Bias | الجزء الرابع: تدريب النموذج واكتشاف التحيز

### 📚 Prerequisites (What You Need First)
-  **Fairness metrics** (from Part 3) - Understanding how to calculate bias
-  **Biased data** (from Part 2) - Having data to analyze

### 🔗 Relationship: What This Builds On
This trains a model and uses our fairness metrics to detect bias!
- Builds on: Fairness metric functions, biased dataset
- Shows: How to detect bias in a trained model

### 📖 The Story
**Before training**: We have data and metrics but no model to analyze.
**After training**: We have a trained model and can detect bias in its predictions!

---

## Step 4: Train Model and Detect Bias | الخطوة 4: تدريب النموذج واكتشاف التحيز

**BEFORE**: We have data and fairness metrics but haven't trained a model yet.

**AFTER**: We'll train a model and use our fairness metrics to detect bias in its predictions!

**Why train a model?** Real-world bias detection happens on trained models, not just data!

---

# Step 4: Train a machine learning model and detect bias in its predictions
# This shows how bias manifests in model predictions

# BEFORE: We have data and metrics but no model predictions
# AFTER: We'll have a trained model and bias detection results

print("\n" + "="*80)
print("🤖 TRAINING MODEL AND DETECTING BIAS")
print("="*80)
print("\nWe'll:")
print("  1. Train a Random Forest classifier on our biased data")
print("  2. Make predictions on test data")
print("  3. Calculate fairness metrics on predictions")
print("  4. Detect bias in the model's behavior\n")

def train_and_analyze_bias(df):
    """
    Train a model and analyze for bias using fairness metrics.
    
    HOW IT WORKS:
    1. Prepare features and target variable
    2. Split data into training and testing sets
    3. Train Random Forest classifier
    4. Make predictions on test set
    5. Calculate demographic parity and equalized odds
    6. Return results for visualization
    
    ⏰ WHEN to use: After having data and fairness metric functions - detect bias in model
    💡 WHY use: Shows how bias in data translates to bias in model predictions
    """
    # Prepare features: Select columns to use for prediction
    X = df[['age', 'experience_years', 'education_level', 'skill_score']]  # Features: Input variables for the model
    y = df['hired']  # Target: What we want to predict (hired or not)
    
    # Split data: Separate into training and testing sets
    # Why split? Training set teaches the model, test set evaluates it
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y  # Split: 70% train, 30% test, stratified to maintain class balance
    )
    
    # Train model: Create and train Random Forest classifier
    # Why Random Forest? Good for classification, handles non-linear relationships
    model = RandomForestClassifier(n_estimators=100, random_state=42)  # Model: 100 trees, fixed random seed
    model.fit(X_train, y_train)  # Train: Learn patterns from training data
    print(" Model trained successfully!")  # Success message: Confirm training complete
    
    # Make predictions: Use model to predict on test data
    y_pred = model.predict(X_test)  # Predictions: Model's predictions for test set
    
    # Add predictions to test data: Combine predictions with test data for analysis
    test_df = X_test.copy()  # Copy: Create copy of test features
    test_df['hired'] = y_test.values  # Actual: Add actual outcomes
    test_df['predicted'] = y_pred  # Predicted: Add model predictions
    test_df['group'] = df.loc[X_test.index, 'group'].values  # Group: Add group labels for fairness analysis
    
    # Calculate bias metrics: Use our fairness functions to detect bias
    parity_rates, parity_disparity = calculate_demographic_parity(test_df)  # Demographic parity: Overall prediction balance
    equalized_odds, tpr_disparity, fpr_disparity = calculate_equalized_odds(test_df)  # Equalized odds: Accuracy balance
    
    return test_df, model, parity_rates, parity_disparity, equalized_odds, tpr_disparity, fpr_disparity  # Return: All results for analysis

# Train model and detect bias
print("Training model and analyzing for bias...")
test_df, model, parity_rates, parity_disparity, equalized_odds, tpr_disparity, fpr_disparity = train_and_analyze_bias(df)

# Print initial results
print("\n📊 BIAS DETECTION RESULTS")
print("="*80)
print("\n1. Demographic Parity")
print("-" * 60)
for group, rate in parity_rates.items():
    print(f"  {group}: {rate:.3f} ({rate*100:.1f}%)")
print(f"\n  Disparity: {parity_disparity:.3f}")
if parity_disparity > 0.1:
    print("  ⚠️  HIGH DISPARITY - Potential bias detected!")
else:
    print("   Low disparity - Fair from demographic parity perspective")

print("\n2. Equalized Odds")
print("-" * 60)
for group, metrics in equalized_odds.items():
    print(f"  {group}:")
    print(f"    TPR: {metrics['TPR']:.3f}")
    print(f"    FPR: {metrics['FPR']:.3f}")
print(f"\n  TPR Disparity: {tpr_disparity:.3f}")
print(f"  FPR Disparity: {fpr_disparity:.3f}")
if tpr_disparity > 0.1 or fpr_disparity > 0.1:
    print("  ⚠️  HIGH DISPARITY - Bias in equalized odds!")
else:
    print("   Low disparity - Fair from equalized odds perspective")
# ============================================================================
# VISUALIZATIONS
# ============================================================================
def visualize_demographic_parity(parity_rates, disparity):
    """Visualize demographic parity"""
    groups = list(parity_rates.keys())
    rates = list(parity_rates.values())
    colors = ['#3498db', '#e74c3c']
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(groups, rates, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
    # Add value labels
    for bar, rate in zip(bars, rates):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{rate:.3f}\n({rate*100:.1f}%)',
               ha='center', va='bottom', fontweight='bold', fontsize=11)
    # Add disparity line
    ax.axhline(y=max(rates), color='red', linestyle='--', alpha=0.5, label='Max')
    ax.axhline(y=min(rates), color='blue', linestyle='--', alpha=0.5, label='Min')
    ax.set_ylabel('Positive Prediction Rate', 
                  fontsize=12, fontweight='bold')
    ax.set_title(f'Demographic Parity Analysis\n'
                f''
                f'Disparity: {disparity:.3f}',
                fontsize=14, fontweight='bold', pad=20)
    ax.set_ylim(0, max(rates) * 1.2)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('demographic_parity.png',
                dpi=300, bbox_inches='tight')
    print(" Saved: demographic_parity.png")
    plt.close()
def visualize_equalized_odds(equalized_odds, tpr_disparity, fpr_disparity):
    """Visualize equalized odds metrics"""
    groups = list(equalized_odds.keys())
    tprs = [equalized_odds[g]['TPR'] for g in groups]
    fprs = [equalized_odds[g]['FPR'] for g in groups]
    x = np.arange(len(groups))
    width = 0.35
    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, tprs, width, label='True Positive Rate (TPR)',
                   color='#2ecc71', alpha=0.8, edgecolor='black')
    bars2 = ax.bar(x + width/2, fprs, width, label='False Positive Rate (FPR)',
                   color='#e74c3c', alpha=0.8, edgecolor='black')
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=9, fontweight='bold')
    ax.set_xlabel('Group', fontsize=12, fontweight='bold')
    ax.set_ylabel('Rate', fontsize=12, fontweight='bold')
    ax.set_title(f'Equalized Odds Analysis\n'
                f''
                f'TPR Disparity: {tpr_disparity:.3f} | FPR Disparity: {fpr_disparity:.3f}',
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(groups)
    ax.legend(fontsize=10)
    ax.set_ylim(0, max(max(tprs), max(fprs)) * 1.2)
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('equalized_odds.png',
                dpi=300, bbox_inches='tight')
    print(" Saved: equalized_odds.png")
    plt.close()
def visualize_confusion_matrices(test_df):
    """Visualize confusion matrices by group"""
    groups = test_df['group'].unique()
    fig, axes = plt.subplots(1, len(groups), figsize=(14, 5))
    for idx, group in enumerate(groups):
        group_data = test_df[test_df['group'] == group]
        cm = confusion_matrix(group_data['hired'], group_data['predicted'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                   cbar_kws={'label': 'Count'})
        axes[idx].set_title(f'{group}\nConfusion Matrix',
                           fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Predicted', fontsize=10)
        axes[idx].set_ylabel('Actual', fontsize=10)
        axes[idx].set_xticklabels(['Not Hired', 'Hired'])
        axes[idx].set_yticklabels(['Not Hired', 'Hired'])
    plt.tight_layout()
    plt.savefig('confusion_matrices_by_group.png',
                dpi=300, bbox_inches='tight')
    print(" Saved: confusion_matrices_by_group.png")
    plt.close()
# ============================================================================
# MAIN EXECUTION
# ============================================================================
if __name__ == "__main__":
    print("="*80)
    print("Unit 2 - Example 1: Bias Detection in ML Models")
    print("")
    print("="*80)
    # Generate data
    print("\n📊 Generating synthetic data with bias...")
    print("")
    df = generate_biased_data(n_samples=2000)
    # Show data summary
    print("\n📋 Data Summary")
    print("-" * 60)
    print(f"Total samples: {len(df)}")
    print(f"Groups: {df["group'].value_counts().to_dict()}')
    print(f"\nHiring rates by group:")
    for group in df['group'].unique():
        rate = df[df['group'] == group]['hired'].mean()
        print(f"  {group}: {rate:.3f} ({rate*100:.1f}%)")
    # Train model and analyze
    print("\n🔍 Training model and analyzing bias...")
    print("")
    test_df, model, parity_rates, parity_disparity, equalized_odds, tpr_disparity, fpr_disparity = train_and_analyze_bias(df)
    # Print results
    print("\n📊 BIAS DETECTION RESULTS")
    print("="*80)
    print("\n1. Demographic Parity")
    print("-" * 60)
    for group, rate in parity_rates.items():
        print(f"  {group}: {rate:.3f} ({rate*100:.1f}%)")
    print(f"\n  Disparity")
    if parity_disparity > 0.1:
        print("  ⚠️  HIGH DISPARITY - Potential bias detected!")
        print("")
    else:
        print("   Low disparity - Fair from demographic parity perspective")
        print("")
    print("\n2. Equalized Odds")
    print("-" * 60)
    for group, metrics in equalized_odds.items():
        print(f"  {group}:")
        print(f"    TPR: {metrics["TPR']:.3f}')
        print(f"    FPR: {metrics["FPR']:.3f}')
    print(f"\n  TPR Disparity")
    print(f"FPR Disparity")
    if tpr_disparity > 0.1 or fpr_disparity > 0.1:
        print("  ⚠️  HIGH DISPARITY - Bias in equalized odds!")
        print("")
    else:
        print("   Low disparity - Fair from equalized odds perspective")
        print("")
    # Create visualizations
    print("\n" + "="*80)
    print("Creating Visualizations")
    print("="*80)
    visualize_demographic_parity(parity_rates, parity_disparity)
    visualize_equalized_odds(equalized_odds, tpr_disparity, fpr_disparity)
    visualize_confusion_matrices(test_df)
    print("\n" + "="*80)
    print(" Example completed successfully!")
    print("")
    print("="*80)
    print("\nKey Takeaways")
    print("1. Multiple fairness metrics can reveal different types of bias")
    print("")
    print("2. Demographic parity and equalized odds measure different aspects")
    print("")
    print("3. It"s important to test for bias before and after model deployment")
    print("")
    print("="*80 + "\n")
